In [4]:
# Written by Jordan Cook 1/18/24 

import pandas as pd 
import numpy as np
import openpyxl
import shutil

In [5]:
# Start by downloading the monthly supplement you want and putting it in the supplement folder 
month = 'september' #NO capitals
year = '2024'
file_path = f"C:/Users/jordancook/OneDrive - Texas A&M University/TAMU Grad School/Research/EIA Data/Supplements/{month.lower()}_generator{year}.xlsx"
supp = pd.read_excel(file_path, skiprows=2)

# Monthly supplement edits 
supp = supp.iloc[:-2]
supp = supp[(supp['Status'] == '(SB) Standby/Backup: available for service but not normally used') |
            (supp['Status'] == '(OP) Operating')]

columns_to_delete = ['Entity ID', 'Entity Name', 'Plant Name', 'Google Map', 'Bing Map', 'Plant State', 'County', 'Balancing Authority Code', 'Sector']
supp = supp.drop(columns=columns_to_delete)

supp = supp.rename(columns={'Plant ID': 'Number of Bus'})

col_name = 'ID'
supp.insert(1, col_name, None)

supp = supp.sort_values(by='Number of Bus')

col_name = 'Primary Label' 
supp.insert(2, col_name, None)

supp['Number of Bus'] = supp['Number of Bus'].astype(int)
supp['Primary Label'] = supp['Number of Bus'].astype(str) + '_' + supp['Generator ID'].astype(str)

supp['ID'] = supp.index % 100

supp = supp.rename(columns={'Prime Mover Code': 'Unit Type Code'})
supp = supp.rename(columns={'Energy Source Code': 'Fuel Type Code'})

supp['Min MW'] = 0
supp['Status'] = 'Yes'
supp['Set Volt '] = 1.0 
supp['Gen MW set point'] = 0 
supp['Max Mvar'] = 0 
supp['Min Mvar'] = 0 
supp['Gen Mvar set point'] = 0 
supp['AGC'] = 'No'
supp['AVR'] = 'No'
supp['MVA Base'] = supp['Nameplate Capacity (MW)']
supp['RegBus Num'] = supp['Number of Bus']

supp['Net Summer Capacity (MW)'] = pd.to_numeric(supp['Net Summer Capacity (MW)'], errors='coerce')
supp['Net Winter Capacity (MW)'] = pd.to_numeric(supp['Net Winter Capacity (MW)'], errors='coerce')
supp['Max MW'] = np.maximum(supp['Net Summer Capacity (MW)'], supp['Net Winter Capacity (MW)'])

columns_to_delete = ['Unit Code', 'Nameplate Capacity (MW)', 'Net Summer Capacity (MW)', 'Net Winter Capacity (MW)', 'Technology']
supp = supp.drop(columns=columns_to_delete)

supp = supp.rename(columns={'Operating Month': 'Operating Date Month'})
supp = supp.rename(columns={'Operating Year': 'Operating Date Year'})
#supp = supp.rename(columns={'Planned Retirement Month': 'Retired Date Month'}) #Don't want these right now 
#supp = supp.rename(columns={'Planned Retirement Year': 'Retired Date Year'})

columns_to_delete = ['Nameplate Energy Capacity (MWh)', 'DC Net Capacity (MW)', 'Planned Derate Year', 'Planned Derate Month', 'Planned Derate of Summer Capacity (MW)', 
                     'Planned Uprate Year', 'Planned Uprate Month', 'Planned Uprate of Summer Capacity (MW)', 'Latitude', 'Longitude']
supp = supp.drop(columns=columns_to_delete)

supp.to_excel(f'{month}{year}_PWready.xlsx', index=False)

#Now adding in the Gen at the top to paste in
workbook = openpyxl.load_workbook(f'{month}{year}_PWready.xlsx')
sheet = workbook.worksheets[0]
sheet.insert_rows(1)
sheet['A1'] = 'Gen'
workbook.save(f'{month}{year}_PWready.xlsx')

# Now you have to delete all PW gens, copy and paste in the Excel file, but don't delete the slack generator

In [6]:
destination_folder = "C:/Users/jordancook/OneDrive - Texas A&M University/TAMU Grad School/Research/EIA Data/Automation Code/Supplements - PW Ready" #CHANGE THIS FOLDER

# Move the file to the destination folder
shutil.move(f'{month}{year}_PWready.xlsx', f'{destination_folder}/{month}{year}_PWready.xlsx')

'C:/Users/jordancook/OneDrive - Texas A&M University/TAMU Grad School/Research/EIA Data/Automation Code/Supplements - PW Ready/september2024_PWready.xlsx'